In [1]:
import numpy as np
import generate_HNF
import spglib
from copy import deepcopy
import permutation
import generate_superlattice

rutile = ([(4, 0, 0),

           (0, 4, 0),

           (0, 0, 3)],

          [(0, 0, 0),

           (0.5, 0.5, 0.5),

           (0.3, 0.3, 0.0),

           (0.7, 0.7, 0.0),

           (0.2, 0.8, 0.5),

           (0.8, 0.2, 0.5)],

          [14, 14, 8, 8, 8, 8])
o_sublattice = ([(4, 0, 0),

                 (0, 4, 0),

                 (0, 0, 3)],

                [(0.3, 0.3, 0.0),

                 (0.7, 0.7, 0.0),

                 (0.2, 0.8, 0.5),

                 (0.8, 0.2, 0.5)],

                [8, 8, 8, 8])
parent_sym = spglib.get_symmetry(rutile)
HNF_list = generate_HNF.generate_all_superlattices(3)
reduced_HNF = generate_HNF.reduce_HNF_list_by_parent_lattice_symmetry(
    HNF_list, parent_sym["rotations"])

In [3]:
# reduced_HNF[1]について
H = reduced_HNF[1]
H_inv = np.linalg.inv(H)

def is_unimodular(M: np.ndarray) -> bool:

    if np.abs(np.around(np.linalg.det(M))) == 1:

        return True

    else:

        return False
def cast_integer_matrix(arr: np.ndarray) -> np.ndarray:

    arr_int = np.around(arr).astype(np.int)

    return arr_int
def is_integer_matrix(matrix):
    a = True
    for i in range(0, 3):
        for j in range(0, 3):
            if not matrix[i][j].is_integer():
                a = False
                break
    return a

for i in range(len(parent_sym["rotations"])):
    check = H_inv.dot(parent_sym["rotations"][i].dot(H))
    if is_integer_matrix(check):
        print(check)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
[[-1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


この方法で得られた部分点群と並進操作を組み合わせてうまくいくか

In [12]:
class Superlattice_spacegroup():
    def __init__(self, parent_sym, HNF):
        self.parent_sym = parent_sym
        self.HNF = HNF
    def print_HNF(self):
        print(self.HNF)

In [13]:
a = Superlattice_spacegroup(parent_sym, reduced_HNF[0])

In [14]:
b = a.print_HNF()

[[1 0 0]
 [0 1 0]
 [0 0 3]]
